In [51]:
<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

SyntaxError: invalid syntax (<ipython-input-51-0c881c342f66>, line 1)

  ---

<center> <font color= #004A94> <font size = 5> Ingeniería Financiera | ITESO </font> <br> <br>

<center> <font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center> <font color=#42c29b><font size=5> Laboratorio 2 , SBG<br> <br> Behavioral Finance en Trading </font> <br> <br>

<center> <font color=#004A94><font size=4> I.F. Juan Francisco Muñoz Elguezabal | franciscome@iteso.mx </font> <br> <br>

  ---

In [ ]:
# Abstract

En este trabajo, vamos a exponer como las herramientas aprendidas en el curso de microestructuras se comportan en un historico de compra y venta de divisas, etefes, entre otros instrumentos financieros.

# Análisis de histórico de cuenta

En este trabajo, el primer paso para poder empezar a analizar un historico es ademas de cargar el archivo, revisar tu informacion "bigdata" para poder limpearla y que sea mas sencillo trabajar con ella.

In [63]:
import datetime
import pandas as pd
import numpy as np
import math as math
import visualizaciones as vs 
from datetime import datetime
from datetime import timedelta
import yfinance as yf 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "browser"


#%% FUNCION: leer archivo
def f_leer_archivo(param_archivo):
    """"
    Parameters
    ----------
    param_archivos : str : nombre de archivo a leer

    Returns
    -------
    df_data : pd.DataFrame : con informacion contenida en archivo leido

    Debugging
    ---------
    param_archivo =  'trading_historico.xlsx'

    """
    # Leer archivo de datos y guardarlo en Data Frame
    df_data = pd.read_excel(param_archivo, sheet_name='Hoja1')

    # Convertir a minusculas el nombre de las columnas
    df_data.columns = [list(df_data.columns)[i].lower() for i in range(0, len(df_data.columns))]
    
    # Asegurar que ciertas columnas son tipo numerico
    numcols =  ['s/l', 't/p', 'commission', 'openprice', 'closeprice', 'profit', 'size', 'swap', 'taxes']
    df_data[numcols] = df_data[numcols].apply(pd.to_numeric)

    return df_data

despues creamos diferentes funciones que nos servirán para un analisis mas a fondo:

 - f_pip_size = Para obtener el número multiplicador y asi expresar la diferencia de precios en pips.
 - f_columnas_tiempos = Para transformar el tiempo.
 - f_columnas_pips = Para saber la cantidad de pips resultantes por cada operación.
 - f_estadisticas_ba = Para saber estadisticas basicas como Operaciones Totales, Operaciones ganadoras.. entre otras.
 - f_estadisticas_ba2 = Para saber la efectividad de trade con respecto a cada instrumento financiero.

In [64]:
#%% FUNCION: 
def f_pip_size(param_ins):
    """"
    Parameters
    ----------
    param_ins : str : nombre de instrumento 

    Returns
    -------
    pips_inst : 

    Debugging
    ---------
    param_ins =  'trading_historico.xlsx'

    """
    
    ""
    
    # encontrar y eliminar un guion bajo
    inst = param_ins.replace('_', '')
    inst = param_ins.replace('-2', '')
    
    # transformar a minusculas
    inst = inst.lower()
    
    #lista de pips por instrumento
    pips_inst =  {'audusd' : 10000,
                  'gbpusd': 10000,
                  'xauusd': 10,
                  'eurusd': 10000,
                  'xaueur': 10,
                  'nas100usd': 10,
                  'us30usd': 10,
                  'mbtcusd':100,
                  'usdmxn': 10000,
                  'eurjpy':10000, 
                  'gbpjpy':10000,
                  'usdjpy':10000,
                  'btcusd':10,
                  'eurgbp':10000,
                  'usdcad':10000,}
    
    return pips_inst[param_ins]

#%% FUNCION:
def f_columnas_tiempo(param_data):

#convertir columna de 'closetime' y 'opentime' utilizando pd.to_datetime
    param_data['closetime'] = pd.to_datetime(param_data['closetime'])
    param_data['opentime'] = pd.to_datetime(param_data['opentime'])
    
    param_data['tiempo'] = [(param_data.loc[i, 'closetime'] - param_data.loc[i, 'opentime']).delta / 1*np.exp(9)
    for i in range(0, len(param_data['closetime']))]
    
    return param_data
    
#%% FUNCION:
def f_columnas_pips(param_data):
    param_data['pips'] = np.zeros(len(param_data['type']))
    for i in range(0,len(param_data['type'])):
        if param_data['type'][i] == 'buy':
            param_data['pips'][i] = (param_data.closeprice[i] - param_data.openprice[i])*f_pip_size(param_ins=param_data['symbol'][i])
        else:
            param_data['pips'][i] = (param_data.openprice[i] - param_data.closeprice[i])*f_pip_size(param_ins=param_data['symbol'][i])
    
    param_data['pips_acum'] = np.zeros(len(param_data['type']))
    param_data['profit_acum'] = np.zeros(len(param_data['type']))    
    param_data['pips_acum'][0] = param_data['pips'][0]
    param_data['profit_acum'][0] = param_data['profit'][0]
            
    for i in range(1,len(param_data['pips'])):
         param_data['pips_acum'][i] = param_data['pips_acum'][i-1] + param_data['pips'][i]
         param_data['profit_acum'][i] = param_data['profit_acum'][i-1] + param_data['profit'][i]
        
    return param_data
    

    
#%% FUNCION:
def f_estadisticas_ba(param_data):
    
    medida = ['Ops totales','Ganadoras','Ganadoras_c','Ganadoras_v','Perdedoras','Perdedoras_c','Perdedoras_v','Media(profit)','Media(pips)',
              'r_efectividad','r_proporcion','r_efectividad_c','r_efectividad_v']
    descripcion = ['Operaciones Totales',
                   'Operaciones Ganadoras',
                   'Operaciones Ganadoras de Compra',
                   'Operaciones perdedoras de Venta',
                   'Operaciones Perdedoras',
                   'Operaciones Perdedoras de Compra',
                   'Operaciones Perdedoras de Venta',
                   'Mediana de Profit de Operaciones', 
                   'Media de Pips de Operaciones',
                   'Ganadoras Totales / Operaciones Totales',
                   'Ganadoras Totales/Perdedoras Totales',
                   'Ganadoras Compras / Operaciones Totales',
                   'Ganadoras Ventas / Operaciones Totales']
   
    #crear uno nuevo lleno de ceros
    zero_data = np.zeros(shape=(len(descripcion),3))
    df_1_tabla = pd.DataFrame(zero_data, columns = ['medida', 'valor', 'descripcion'])
    #resolver las descripciones dadas 
    df_1_tabla['valor'][0] = len(param_data['profit'])
    df_1_tabla['valor'][1] = param_data['profit'].gt(0).sum()
    x = 0
    for i in range(0,len(param_data['type'])): 
        if param_data['type'][i] == 'buy' and param_data['profit'][i] > 0 :
            x = x+1
    df_1_tabla['valor'][2] = x
    
    x = 0
    for i in range(0,len(param_data['type'])): 
        if param_data['type'][i] == 'sell' and param_data['profit'][i] > 0 :
            x = x+1
    df_1_tabla['valor'][3] = x
    df_1_tabla['valor'][4] = df_1_tabla['valor'][0] - df_1_tabla['valor'][1]
    x = 0
    for i in range(0,len(param_data['type'])): 
        if param_data['type'][i] == 'buy' and param_data['profit'][i] < 0 :
            x = x+1
    df_1_tabla['valor'][5] = x
    
    x = 0
    for i in range(0,len(param_data['type'])): 
        if param_data['type'][i] == 'sell' and param_data['profit'][i] < 0 :
            x = x+1
    df_1_tabla['valor'][6] = x
    df_1_tabla['valor'][7] = param_data.profit.median()
    df_1_tabla['valor'][8] = param_data.pips.median()
    df_1_tabla['valor'][9] = df_1_tabla['valor'][1] / df_1_tabla['valor'][0]
    df_1_tabla['valor'][10] = df_1_tabla['valor'][1] / df_1_tabla['valor'][4]
    df_1_tabla['valor'][11] = df_1_tabla['valor'][2] / df_1_tabla['valor'][0] 
    df_1_tabla['valor'][12] = df_1_tabla['valor'][3] / df_1_tabla['valor'][0] 
    
    for i in range(0,len(medida)):
        df_1_tabla['medida'][i] = medida[i]
        df_1_tabla['descripcion'][i] = descripcion[i]
        
    return df_1_tabla

#%%def f_estadisticas_ba2(param_data):
    


def f_estadistica_ba2(param_data):

    #crear un arreglo con los symbolos que se uilizaran para el ranking
    symbol = ['xauusd',
              'eurusd',
              'xaueur',
              'bcousd',
              'cornusd',
              'mbtcusd',
              'wticousd',
              'spx500usd',
              'audusd', 
              'gbpusd',
              'xaueur', 
              'nas100usd',
              'usdmxn',
              'eurjpy',
              'gbpjpy', 
              'usdjpy',
              'btcusd',
              'eurgbp', 
              'usdcad']
    
    columns = {'symbol':np.zeros(len(symbol)),
                   'rank':np.zeros(len(symbol))}
    
    df_1_ranking = pd.DataFrame(columns)
    
    #
    for i in range(len(df_1_ranking)):
        f0 = param_data['symbol']==symbol[i]
        f1 = param_data[f0]
        if f1.empty == False:
            f1 = param_data[f0]
            f2 = param_data[(param_data['symbol']==symbol[i]) & (param_data['profit']>=0)]            
            df_1_ranking['rank'][i]=round(len(f2)/len(f1),4)*100
            df_1_ranking['symbol'][i]=symbol[i]
        else:
            df_1_ranking['rank'][i]="nan"
        pass
   
    df_1_ranking =df_1_ranking.sort_values(by=['rank'],ascending=[False])
    
    df_1_ranking =df_1_ranking.dropna()
 
    


    return df_1_ranking





In [65]:


datos = f_leer_archivo(param_archivo='archivo_tradeview_1.xlsx')
f_pip_size(param_ins='audusd')
datos = f_columnas_tiempo(param_data=datos)
datos = f_columnas_pips(param_data=datos)
df_1tabla = f_estadisticas_ba(param_data=datos)
df_1ranking = f_estadistica_ba2(param_data=datos)

print(datos)

       order            opentime  type  size  symbol   openprice       s/l  \
0   24316034 2019-08-27 06:57:50   buy  0.04  xauusd  1529.21000  1523.000   
1   24334215 2019-08-27 17:06:20   buy  0.01  eurjpy   117.66400   116.950   
2   24342155 2019-08-27 19:25:39   buy  0.05  eurjpy   117.34600   116.950   
3   24373260 2019-08-28 17:11:33   buy  0.01  eurusd     1.10791     1.104   
4   24392415 2019-08-29 06:35:27   buy  0.01  gbpusd     1.22024     1.208   
..       ...                 ...   ...   ...     ...         ...       ...   
79  25148355 2019-09-24 20:09:52  sell  0.06  xauusd  1532.02000  1538.000   
80  25148480 2019-09-24 20:11:20  sell  0.02  xauusd  1531.50000  1538.000   
81  25148522 2019-09-24 20:12:41  sell  0.01  xauusd  1531.86000  1538.000   
82  25159294 2019-09-25 01:53:47  sell  0.06  xauusd  1531.06000  1537.000   
83  25159303 2019-09-25 01:54:46  sell  0.01  xauusd  1531.28000  1537.000   

          t/p           closetime  closeprice  commission  taxe

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning:


A value is tryi

In [66]:
print(df_1tabla)

             medida      valor                              descripcion
0       Ops totales  84.000000                      Operaciones Totales
1         Ganadoras  46.000000                    Operaciones Ganadoras
2       Ganadoras_c  20.000000          Operaciones Ganadoras de Compra
3       Ganadoras_v  26.000000          Operaciones perdedoras de Venta
4        Perdedoras  38.000000                   Operaciones Perdedoras
5      Perdedoras_c  19.000000         Operaciones Perdedoras de Compra
6      Perdedoras_v  19.000000          Operaciones Perdedoras de Venta
7     Media(profit)   1.205000         Mediana de Profit de Operaciones
8       Media(pips)   5.100000             Media de Pips de Operaciones
9     r_efectividad   0.547619  Ganadoras Totales / Operaciones Totales
10     r_proporcion   1.210526     Ganadoras Totales/Perdedoras Totales
11  r_efectividad_c   0.238095  Ganadoras Compras / Operaciones Totales
12  r_efectividad_v   0.309524   Ganadoras Ventas / Operaciones 

In [67]:
print(df_1ranking)

    symbol    rank
16  btcusd  100.00
17  eurgbp  100.00
12  usdmxn   80.00
0   xauusd   73.68
9   gbpusd   66.67
15  usdjpy   60.00
1   eurusd   47.83
8   audusd   33.33
13  eurjpy   28.57
18  usdcad   28.57
14  gbpjpy   25.00


Se utilizo la tabla de ranking para crear una graphicpie y asi hacer mas vizual el resultado.
* la graphica se proyecta en una ventana del buscador ya que la version de spyder con la que trabaje no soporta la proyeccion de graphicas.

In [68]:
#%%grafica del ranking abierta en browser ya que al parecer mi version de spyder no esta actualizada y no se podian ver aqui mismo
graf = f_estadistica_ba2(param_data=datos)
df = pd.DataFrame(graf)


# pull is given as a fraction of the pie radius
fig = go.Figure()
labels = df['symbol']
values = df['rank']
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0.2, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])])
fig.update_layout(title = "Efectividad", font = dict(size = 20))
py.iplot(fig)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:180: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:181: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:183: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Medidas de atribución al desempeño

Despues se realizo las medidas de atribución al desempeño en ellas se encuentran funciones para saber el capital acumulado despues de que los trades se hayan echo,  funciones para saber la ganancia diaria y el capital acumulado diario y finalmente unas funciones para saber estadisticas generales como sharpe', 'sortino_c',  'sortino_v',  'drawdown_capi', 'drawup_capi',  'information_r'.

In [73]:
#%% funcio de capital acumulado empezando com capital de 5000

def f_columnas_capital_acum(param_data):
                                
    param_data['capital_acum'] = np.zeros(shape=(len(param_data['profit']),1))
    
    
    initial_investment = 5000
    for i, r in enumerate(param_data['profit']):
        param_data['capital_acum'][i] = initial_investment + r
        initial_investment = initial_investment + r

    



   

    return param_data  




#%%crear una columna con los rendimientos logaritmicos
def f_columnas_log(param_data):
    
    param_data['rendimiento_log'] = np.zeros(len(param_data['type']))
    param_data['rendimiento_log'][0] = math.log(param_data['capital_acum'][0] / 5000)
    
      
    for i in range(1,len(param_data['pips'])):
         param_data['rendimiento_log'][i] = math.log(param_data['capital_acum'][i] / param_data['capital_acum'][i-1])
    
    return param_data


#%%
    

def f_profit_diario(param_archivo):
    #tuve que volver  descargar el excel por problemas de fechas, lo senti mas facil haciendolo desde cero
    d = pd.read_excel(param_archivo, sheet_name='Hoja1')

     
    
#aplicamos to_datetime
    d['closeTime'] = pd.to_datetime(d['closeTime'].copy()).dt.date
#creamos el dataframe con las columnas
    trades = pd.DataFrame(columns=['timestamp', 'profit_d', 'profit_acum_d'])
#timesstamp lo hice de los closetime, ya que habia intentado con opentime peor no daba el profit diario realmente
    trades['timestamp'] = d['closeTime']
    trades['profit_d'] = d['Profit']
    trades['profit_acum_d'] = np.zeros(shape=(len(trades),1))
    #sumas los close time que sean iguales  ... para que sea diario, termina siendo timestamp el index
    trades = trades.groupby(['timestamp']).sum()
#creas el profit acumulado diario
    initial_investment = 5000
    for i, r in enumerate(trades['profit_d']):
        trades['profit_acum_d'][i] = initial_investment + r
        initial_investment = initial_investment + r

    return trades
     

    


#%%para sacar sortinos y sharps equidistantes
    

def f_profit_diario_eq(param_archivo):
    #tuve que volver  descargar el excel por problemas de fechas, lo senti mas facil haciendolo desde cero
    d = pd.read_excel(param_archivo, sheet_name='Hoja1')

     
    
#aplicamos to_datetime
    d['closeTime'] = pd.to_datetime(d['closeTime'].copy()).dt.date
    
#creamos el dataframe con las columnas
    trades = pd.DataFrame(columns=['timestamp', 'profit_d', 'profit_acum_d',
                                   'profit_d_c', 'profit_d_v',
                                   'profit_acum_d_v','profit_acum_d_c',
                                   'rendimiento_log'])
#timesstamp lo hice de los closetime, ya que habia intentado con opentime peor no daba el profit diario realmente
    trades['timestamp'] = d['closeTime']
    trades['profit_d'] = d['Profit']
    trades['profit_acum_d'] = np.zeros(shape=(len(trades),1))
    trades['rendimiento_log'] = np.zeros(shape=(len(trades),1))
    trades['profit_acum_d_c'] = np.zeros(shape=(len(trades),1))
    trades['profit_acum_d_v'] = np.zeros(shape=(len(trades),1))
    trades['profit_d_c'] = np.zeros(shape=(len(trades),1))
    trades['profit_d_v'] = np.zeros(shape=(len(trades),1))
    #compra
    for i in range(0,len(d['Type'])): 
        if d['Type'][i] == 'buy' :
            trades['profit_d_c'][i] = d['Profit'][i]
        else:
            trades['profit_d_c'][i] = 0
            
         
    trades['profit_acum_d_c'] = np.zeros(shape=(len(trades),1))
    #venta
    
    for i in range(0,len(d['Type'])): 
        if d['Type'][i] == 'sell' :
            trades['profit_d_v'][i] = d['Profit'][i]
        else:
            trades['profit_d_v'][i] = 0
         
    
    
    
    #juntos por timestamp
    trades = trades.groupby(['timestamp']).sum()
    
    initial_investment = 5000
    ##compr
    for i, r in enumerate(trades['profit_d_c']):
        trades['profit_acum_d_c'][i] = initial_investment + r
        initial_investment = initial_investment + r
    
    ##venta
    initial_investment = 5000
    for i, r in enumerate(trades['profit_d_v']):
        trades['profit_acum_d_v'][i] = initial_investment + r
        initial_investment = initial_investment + r
    
    ##acum
    initial_investment = 5000
    for i, r in enumerate(trades['profit_d']):
        trades['profit_acum_d'][i] = initial_investment + r
        initial_investment = initial_investment + r
    

    
###############
    for i in range(0,len(trades['profit_d'])):
         trades['rendimiento_log'][i] = math.log(trades['profit_acum_d'][i] / trades['profit_acum_d'][i-1])
    
    
 
    return trades
    
    
    
    
     

    

#%%    
def f_estadisticas_mad(param_data):
    
    #termine descargando los rendimientos del etef "MSFT"=Microsoft
    #NASDAQ:  ya que no pude poner la funcion de oanda  que nos dio el profe
    msft = yf.Ticker("MSFT")
    x = pd.DataFrame(msft.dividends)
    x = x.iloc[20,:]
    rf = .08
    m = 0.3/300
    
    medidas = ['sharpe','sortino_c','sortino_v', 'drawdown_capi','drawdown_capi',
               'information_r']
    descripcion = ['Sharpe Ratio', 'Sortino Ratio para Posiciones de Compra','Sortino Ratio para Posiciones de Venta', 'DrawDown de Capital',
                   'DrawUp de Capital','Information Ratio']
                              
                              
      
   
    zero_data = np.zeros(shape=(len(descripcion),3))
    df_estadisticas_mad = pd.DataFrame(zero_data, columns = ['medidas', 
                                                             'valor', 
                                                             'descripcion'])
    for i in range(0,len(medidas)):
        df_estadisticas_mad['medidas'][i] = medidas[i]
        df_estadisticas_mad['descripcion'][i] = descripcion[i]

    #medidas 0 sharp
    salida = np.log(param_data.profit_acum_d / param_data.profit_acum_d.shift()).dropna()
    sum_salid = np.sum(salida)
    std_salid = salida.std()
    df_estadisticas_mad.valor[0] = (sum_salid - rf) / std_salid
    
 
    
    
# medidas 1,2 sortino

    salida2 = np.log(param_data.profit_acum_d_c[1:].values / param_data.profit_acum_d_c[:-1].values)

    de = salida2 - m
    de[de > 0] = 0

    df_estadisticas_mad.valor[1] = (salida2.mean() - m) / (((de*2).mean())*0.5)
    

    # Sortino venta
 

    salida3 = np.log(param_data.profit_acum_d_v[1:].values / param_data.profit_acum_d_v[:-1].values)
    
    de2 = salida3 - m
    de2[de2 > 0] = 0
    
    df_estadisticas_mad.valor[2] = (salida3.mean() - m) / ((de*2).mean())*0.5
    
    

    

   # medidas 3 drawdown_capi_c
    df_estadisticas_mad.valor[3] =  param_data.profit_acum_d.min()

    # medidas 4 drawup_capi_v
    df_estadisticas_mad.valor[4] =  param_data.profit_acum_d.max()







    


    # information_r
    

      
  
    profit_prom = param_data.rendimiento_log.mean()
   
    # Promedio de rendimientos del Microsoft NASDAQ: MSFT
    bench_prom = x.mean()
    
    num_ir = profit_prom - bench_prom

    # Denominador Information Ratio
    # Diferencia por rows de los rendimientos del profit_acm_d y del Microsoft NASDAQ: MSFT
    #dif_denom = param_data['profit_acum_d'].mean() - x.mean()
 
    
    #dif_denom = param_data.rendimiento_log - x
    #denom_ir = dif_denom.std()
    

    df_estadisticas_mad.valor[5] = num_ir#/denom_ir
    
    
    
    
    
    
    
    return df_estadisticas_mad

    
    



In [74]:



diario = f_profit_diario(param_archivo='archivo_tradeview_1.xlsx')
#convertir todos los datos para que sean equidistantes
equidistante = f_profit_diario_eq(param_archivo='archivo_tradeview_1.xlsx')


estadisticas_mad = f_estadisticas_mad(param_data=equidistante)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:109: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:107: SettingWithCopyWarning:


A value is tr

         medidas        valor                              descripcion
0         sharpe    -3.569475                             Sharpe Ratio
1      sortino_c     0.212144  Sortino Ratio para Posiciones de Compra
2      sortino_v    -0.089819   Sortino Ratio para Posiciones de Venta
3  drawdown_capi  4842.130000                      DrawDown de Capital
4  drawdown_capi  5263.960000                        DrawUp de Capital
5  information_r    -0.130000                        Information Ratio


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:177: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:176: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:183: SettingWithCopyWarning:


A value i

In [75]:
print(diario)

            profit_d  profit_acum_d
timestamp                          
2019-08-27     27.61        5027.61
2019-08-28      5.00        5032.61
2019-08-29     -1.52        5031.09
2019-08-30    -17.16        5013.93
2019-09-02    -33.90        4980.03
2019-09-03   -137.90        4842.13
2019-09-04     29.11        4871.24
2019-09-06      7.27        4878.51
2019-09-11     44.24        4922.75
2019-09-12     -7.61        4915.14
2019-09-13     64.68        4979.82
2019-09-16    106.68        5086.50
2019-09-17     -2.64        5083.86
2019-09-18     13.62        5097.48
2019-09-19     32.85        5130.33
2019-09-20     14.46        5144.79
2019-09-24     50.29        5195.08
2019-09-25     68.88        5263.96
2019-09-26    -16.67        5247.29
2019-09-27      6.84        5254.13


In [78]:
print(equidistante)

            profit_d  profit_acum_d  profit_d_c  profit_d_v  profit_acum_d_v  \
timestamp                                                                      
2019-08-27     27.61        5027.61       17.92        9.69          5009.69   
2019-08-28      5.00        5032.61        5.00        0.00          5009.69   
2019-08-29     -1.52        5031.09       -1.52        0.00          5009.69   
2019-08-30    -17.16        5013.93      -37.57       20.41          5030.10   
2019-09-02    -33.90        4980.03      -33.64       -0.26          5029.84   
2019-09-03   -137.90        4842.13      -51.96      -85.94          4943.90   
2019-09-04     29.11        4871.24        0.00       29.11          4973.01   
2019-09-06      7.27        4878.51        0.00        7.27          4980.28   
2019-09-11     44.24        4922.75       19.86       24.38          5004.66   
2019-09-12     -7.61        4915.14      -29.39       21.78          5026.44   
2019-09-13     64.68        4979.82     

In [77]:
print(estadisticas_mad)

         medidas        valor                              descripcion
0         sharpe    -3.569475                             Sharpe Ratio
1      sortino_c     0.212144  Sortino Ratio para Posiciones de Compra
2      sortino_v    -0.089819   Sortino Ratio para Posiciones de Venta
3  drawdown_capi  4842.130000                      DrawDown de Capital
4  drawdown_capi  5263.960000                        DrawUp de Capital
5  information_r    -0.130000                        Information Ratio


Se utilizo la tabla de estadisticas_mad para crear una grafico y asi hacer mas vizual el resultado.
* la graphica se proyecta en una ventana del buscador ya que la version de spyder con la que trabaje no soporta la proyeccion de graphicas.

In [43]:

grafs = diario
graf1 = pd.DataFrame(grafs)


fig2 = go.Figure()
fig2.add_trace(go.Scatter(x= graf1.index, y = graf1.profit_acum_d, name = 'profit acumulado', mode = 'lines', marker = dict(color = 'Black')))
fig2.add_trace(go.Scatter(x = graf1.index, y =  [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5195,None,5263.96,None] , name = 'drawup',connectgaps=True , mode = 'lines', line={'dash': 'dash', 'color': 'green'}))
fig2.add_trace(go.Scatter(x = graf1.index, y = [None,None,None,None,5055.64,None,4843.7,None,None,None,None,None,None,None,None,None,None,None,None,None], name = 'drawdown', connectgaps=True, mode = 'lines', line={'dash': 'dash', 'color': 'red'}))

fig2.update_layout(title = "DrawDown y DrawUp", xaxis_title = "Time", yaxis_title = "Profit")
    
py.iplot(fig2)
